In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import json
import requests
from urllib.request import urlopen
import urllib.request
from datetime import datetime
from tqdm import tqdm
import time
import numpy as np
from concurrent.futures import ThreadPoolExecutor, as_completed

In [2]:
hrefs = []
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'} 

In [14]:
r = requests.get("https://thediagnosa.com/jenis-penyakit/",headers = headers)
soup = BeautifulSoup(r.content, "html.parser")
a = soup.find('div', attrs = {"class":"main-navigation ast-inline-flex"})
links = [x.find('a').get('href') for x in a.find_all('li')]
links

['https://thediagnosa.com/jenis-penyakit/',
 'http://thediagnosa.com/jenis-penyakit/',
 'https://thediagnosa.com/covid-19-simptom-punca-dan-rawatan/',
 'https://thediagnosa.com/coronavirus-simptom-punca-rawatan/',
 'https://thediagnosa.com/kencing-manis-punca-simptom-rawatan/',
 'https://thediagnosa.com/sakit-jantung-punca-simptom-rawatan/',
 'https://thediagnosa.com/tekanan-darah-tinggi-punca-rawatan/',
 'https://thediagnosa.com/simptom-dan-tanda-tanda/',
 'http://thediagnosa.com/simptom-dan-tanda-tanda/',
 'https://thediagnosa.com/sesak-nafas-punca-rawatan-penyiasatan/',
 'https://thediagnosa.com/batuk-punca-dan-jenis-ubat-batuk/',
 'https://thediagnosa.com/pening-kepala-punca-tanda-rawatan/',
 'https://thediagnosa.com/kebas-tangan-punca-dan-cara-rawatan/',
 'http://thediagnosa.com/temujanji/',
 'http://thediagnosa.com',
 'https://thediagnosa.com/penafian/',
 'http://thediagnosa.com/menjadi-penulis/']

In [16]:
max_worker = 5


for url in links:

    while True:
        try:
            r = requests.get(url,headers = headers)
            break
        except Exception as e:
            print(e)
            time.sleep(1.0)

    soup = BeautifulSoup(r.text, "html.parser")
  
    for link in soup.find_all('a'):

        href = link.get('href')

        if href not in links:
            hrefs.append(href)

    

In [18]:
hrefs = list(set(hrefs))

In [19]:
hrefs

['http://thediagnosa.com/tangan-berpeluh-punca-cara-rawatan/',
 'http://thediagnosa.com/multiple-sclerosis-punca-simptom-dan-rawatan/',
 'https://thediagnosa.com/cirit-birit-punca-cara-rawatan/',
 'http://thediagnosa.com/air-ketuban-banyak-punca-tanda-rawatan/',
 'http://thediagnosa.com/tercekik-tulang-ikan-apa-yang-perlu-dibuat-dan-rawatan/',
 'http://thediagnosa.com/bernafas-laju-punca-rawatan/',
 'http://thediagnosa.com/angin-ahmar-punca-simptom-rawatan/',
 'https://thediagnosa.com/sakit-tapak-kaki-punca-dan-rawatan/',
 'http://thediagnosa.com/ruam-panas-punca-simptom-dan-rawatan/',
 'http://thediagnosa.com/hilang-deria-bau-anosmia-punca-simptom-rawatan/',
 'http://thediagnosa.com/kekurangan-vitamin-a-punca-simptom-dan-rawatan/',
 'http://thediagnosa.com/sakit-pergelangan-tangan-punca-simptom-rawatan/',
 'http://thediagnosa.com/batuk-berdarah-punca-rawatan/',
 'http://thediagnosa.com/kencing-berbuih-punca-dan-apa-yang-perlu-dibuat/',
 'https://thediagnosa.com/batuk-berpanjangan-punc

In [5]:
len(hrefs)

500

In [21]:
with open(f'herald-link.json', 'a') as f:
    json.dump(hrefs, f)

In [3]:
with open(f'herald-link.json') as f:
    hrefs = json.load(f)

In [10]:
def process_url(x):
    
    while True:
        try:
            r = requests.get(x, headers=headers)
            
            if r.status_code == 508:
                time.sleep(20)
                continue
            if r.status_code ==404:
                return None
                
            break
        except Exception as e:
            print(e)
            time.sleep(5.0)
            return None
    
    soup = BeautifulSoup(r.text, "html.parser")

    try:
        headline = soup.find('h1',attrs={'class':'entry-title'}).text
        block = soup.find('div',attrs={'class':'entry-content clear'})
        p = [p.text for p in block.find_all() if p.text and len(p.text.split()) > 5]
    except Exception as e:
        print('error in link:'+ x)
        print(e)
        return None

    
    data = {'url': x,'headline': headline, 'content': p}
    
    return data

In [11]:
process_url(hrefs[0])

{'url': 'http://thediagnosa.com/tangan-berpeluh-punca-cara-rawatan/',
 'headline': 'Tangan Berpeluh: Punca dan Cara Rawatan',
 'content': ['Berpeluh merupakan sesuatu yang biasa bagi seseorang manusia, namun keadaan berpeluh boleh menjadi satu masalah yang serius sekiranya didapati berlebihan terutama pada bahagian tangan dan ia dikenali sebagai hyperhidrosis. Masalah seperti ini boleh menganggu aktiviti harian bagi mereka yang mengalaminya terutama bagi masalah tangan berpeluh.',
  'Bagi golongan kanak-kanak atau remaja, masalah tangan berpeluh boleh menyebabkan pretasi pembelajaran mereka di sekolah akan terganggu.',
  'Hyperhidrosis: Adalah masalah berpeluh yang berlebihanSaraf sympathetic:\xa0Saraf yang berfungsi menyediakan badan seseorang untuk berada dalam keadaan sedia untuk melawan atau melarikan diriPrognosis:\xa0Adalah sesuatu kesudahan bagi pesakit',
  'Masalah tangan berpeluh atau dikenali sebagai Hyperhidrosis adalah masalah dimana seseorang individu menghasilkan peluh ya

In [ ]:
max_workers = 3

with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = [executor.submit(process_url, x) for x in hrefs]
    
    for future in tqdm(futures, total=len(hrefs)):
        result = future.result()
        if result:
            with open('diagnosa-6.jsonl', 'a') as f:
                json.dump(result, f)
                f.write('\n')

In [15]:
single_page = ['https://thediagnosa.com/covid-19-simptom-punca-dan-rawatan/',
 'https://thediagnosa.com/coronavirus-simptom-punca-rawatan/',
 'https://thediagnosa.com/kencing-manis-punca-simptom-rawatan/',
 'https://thediagnosa.com/sakit-jantung-punca-simptom-rawatan/',
 'https://thediagnosa.com/tekanan-darah-tinggi-punca-rawatan/',
 'https://thediagnosa.com/sesak-nafas-punca-rawatan-penyiasatan/',
 'https://thediagnosa.com/batuk-punca-dan-jenis-ubat-batuk/',
 'https://thediagnosa.com/pening-kepala-punca-tanda-rawatan/',
 'https://thediagnosa.com/kebas-tangan-punca-dan-cara-rawatan/',
 'http://thediagnosa.com/temujanji/',
 'http://thediagnosa.com/menjadi-penulis/']

In [29]:
process_url(single_page[5])

{'url': 'https://thediagnosa.com/sesak-nafas-punca-rawatan-penyiasatan/',
 'headline': 'Sesak Nafas: Punca, Cara Rawatan dan Penyiasatan',
 'content': ['Apakah punca kepada masalah sesak nafas berserta rawatannya? Artikel ini akan menerangkan dengan lengkap berkenaan punca-punca kepada masalah ini berserta rawatan sesak nafas.',
  '  Isi Kandungan   Pengenalan Punca Sesak Nafas  Masalah Paru-Paru Masalah Jantung Masalah Perut   Simptom Selain Sesak Nafas Ujian Pengesanan Punca Rawatan  ',
  ' Pengenalan Punca Sesak Nafas  Masalah Paru-Paru Masalah Jantung Masalah Perut   Simptom Selain Sesak Nafas Ujian Pengesanan Punca Rawatan ',
  'Punca Sesak Nafas  Masalah Paru-Paru Masalah Jantung Masalah Perut  ',
  ' Masalah Paru-Paru Masalah Jantung Masalah Perut ',
  'Mereka yang mengalami sesak nafas selalunya akan mempunyai pengalaman yang berbeza. Akan ada yang mengatakan susah untuk tarik nafas, ada pula yang mengatakan dadanya ditekan kuat hingga menyusahkan ia untuk menarik nafas dan ada

In [28]:
for x in single_page:
    result = process_url(x)

    with open('diagnosa-6.jsonl', 'a') as f:
        json.dump(result, f)
        f.write('\n')